# Lab 4: Divide and Conquer Algorithms

> Date: 11/08/2024
> Author: Adam Haile, Leigh Goetsch, Aiden Miller
> Class: CSC3310
>
> ### Learning Outcomes
>
> - Design an algorithm for a given computational problem statement
> - Justify the correctness of an algorithm
> - Perform asymptotic complexity analysis of the run time of an algorithm
> - Generate test cases for an algorithm
> - Correctly implement an algorithm from pseudocode
> - Design and execute benchmarks for an algorithm



## Proposed Solution - Quickselect Algorithm
The quickselect algorithm is similar to the quicksort algorith, but with modifications so that it instead returns the $k^{th}$ smallest value in lstay, rather than a sorted lstay.

In [9]:
import random

In [ ]:
def quickselect(lst, k):

    def partition(low, high):
        pivot_index = random.randint(low, high)
        lst[pivot_index], lst[high] = lst[high], lst[pivot_index]
        pivot = lst[high]
        
        i = low
        for j in range(low, high):
            if lst[j] < pivot:
                lst[i], lst[j] = lst[j], lst[i]
                i += 1
        lst[i], lst[high] = lst[high], lst[i]
        return i

    def select(low, high, k_smallest):
        if low == high:
            return lst[low]
        
        pivot_index = partition(low, high)
        
        if k_smallest == pivot_index:
            return lst[k_smallest]
        elif k_smallest < pivot_index:
            return select(low, pivot_index - 1, k_smallest)
        else:
            return select(pivot_index + 1, high, k_smallest)

    return select(0, len(lst) - 1, k - 1)